# Storytelling Data Visualization Lab

In this lab you'll use a dataset called `housing_prices.csv` which contains the sales data of houses. The dataset and descriptions of the columns are available from [Kaggle](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data). For your convenience, you can review the descriptions of the data columns from [here](https://drive.google.com/file/d/1-iXooLjNuEXU41dqz8ORQ5JEZPHd9x0X/view?usp=sharing).

Pretend you are a data analyst at an investment company where the board decided to make investments in real estates. Your boss asked you to analyze this housing sales dataset and present to the investment managers on **what features of houses are strong indicators of the final sale price**. You need to present your findings in intuitive ways so that the investment managers understand where your conclusions come from.

#### You will use the appropriate data visualization graphs to tell your stories.

## Challenge 1 - Understanding the Dataset

After receiving the data and clarifying your objectives with your boss, you will first try to understand the dataset. This allows you to decide how you will start your research in the next step.

The dataset is [here](https://drive.google.com/file/d/1MRhRtdX8QuPPEhelBIS_FEl5vJjRLSeE/view?usp=sharing). Please download it and place it in the data folder.<br>
First, import the basic libraries and the dataset.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
df = pd.read_csv('../Data/housing_prices.csv')
#Paolo: I think it is a good idea to split importing dataset abd library in different cells

#### As a routine before analyzing a dataset, print the first few rows of the dataset

In [ ]:
df.head()

You find the dataset has 81 columns which are a lot. 

#### Since the column `Id` is meaningless in our data visualization work, let's drop it

In [ ]:
# your code here
df.drop(labels='Id', axis=1, inplace=True)

You care about missing values. If a column has too many missing values, it is not reliable to use it to predict sales price.

#### In the cell below, calculate the percentage of missing values for each column. 

Make a table containing the column name and the percentage of missing values. Print the columns where more than 20% of values are missing. An example of what your output  should look like is [here](https://drive.google.com/file/d/1cuq6qhFZC5wavm-_STcxktBKdAc4xvH8/view?usp=sharing)

[This reference](https://stackoverflow.com/questions/51070985/find-out-the-percentage-of-missing-values-in-each-column-in-the-given-dataset) can help you make the missing values table.

In [ ]:
# your code here
percent_missing = df.isnull().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'column_name': df.columns,
                                 'percent_missing': percent_missing})
missing_value_df.sort_values('percent_missing', inplace=True)
missing_value_df=missing_value_df[missing_value_df['percent_missing']>20]
missing_value_df

#### Drop the columns you find that have more than 20% missing values.

After dropping, check the shape of your dataframes. You should have 75 columns now.

In [ ]:
# your code here
df.drop(labels=['FireplaceQu','Fence','Alley','MiscFeature','PoolQC'], axis=1, inplace=True)

In [ ]:
df

Since you're asked to analyze sale prices, first let's see if the sale prices (column `SalePrice`) has a normal distribution. This is important because normally distributed data can be better represented with mathematical models.

#### In the cell below, use the propriate graph to visualize the shape of distribution of the sale prices. Then explain what you find from the graph about data distribution.

In [ ]:
# your code here
sns.distplot(df['SalePrice'])

In [ ]:
# your comment here
#There seems to be a normal distribution with a positive skew since the right tail is longer.
#This distribution is right-skewed. The mean is more to the right than the median. 
#Paolo: good observation

## Bonus Challenge 1 - Adjust Data Distribution

If you used the correct method in the previous step, you should have found the data distribution is skewed to the left. In order to improve your data visualization in the next steps, you can opt to adjust the `SalePrice` column by applying a mathematical function to the values. The goal is to produce a bell-shape normal distribution after applying the mathematical function to the sale price.

*This technique is optional in data visualization but you'll find it useful in your future machine learning analysis.*

#### In the cell below, adjust the `SalePrice` column so that the data are normally distributed.

Try applying various mathematical functions such as square root, power, and log to the `SalePrice` column. Visualize the distribution of the adjusted data until you find a function that makes the data normally distributed. **Create a new column called `SalePriceAdjusted` to store the adjusted sale price.**

[This reference](https://trainingdatascience.com/workshops/histograms-and-skewed-data/) shows you examples on how to adjust skewed data.

In [ ]:
# your code here


## Challenge 2 - Exploring Data with Common Sense

Now that we have a general understanding of the dataset, we start exploring the data with common sense by means of data visualization. Yes, in data analysis and even machine learning you are often required to use common sense. You use your common sense to make a scientific guess (i.e. hypothesis) then use data analytics methods to test your hypothesis.

This dataset is about housing sales. According to common sense, housing prices depend on the following factors:

* **Size of the house** (`GrLivArea`, `LotArea`, and `GarageArea`).

* **Number of rooms** (`BedroomAbvGr`, `KitchenAbvGr`, `FullBath`, `HalfBath`, `BsmtFullBath`, `BsmtHalfBath`).

* **How long the house has been built or remodeled** (`YearBuilt` and `YearRemodAdd`).

* **Neighborhood of the house** (`Neighborhood`).

#### In this challenge, use the appropriate graph type to visualize the relationships between `SalePrice` (or `SalePriceAdjusted`) and the fields above. 

Note that:

* Transform certain columns in order to visualize the data properly based on common sense. For example:
    * Visualizing how the number of half bathrooms affected the sale price probably does not make sense. You can create a new column to calculate the total number of bathrooms/rooms then visualize with the calculated number.
    * `YearBuilt` and `YearRemodAdd` are year numbers not the age of the house. You can create two new columns for how long the house has been built or remodeled then visualize with the calculated columns.
* Make comments to explain your thinking process.

In [ ]:
# your code here
# add cells as needed
df['Total Area']=df['GrLivArea']+df['LotArea']+ df['GarageArea']
df['Total Rooms']=df['BedroomAbvGr'] + df['KitchenAbvGr'] + df['FullBath'] + df['HalfBath'] + df['BsmtFullBath'] + df['BsmtHalfBath']
df['Total Bathrooms'] = df['FullBath'] + df['HalfBath'] + df['BsmtFullBath'] + df['BsmtHalfBath']
df['Total Years Built']=2020 - df['YearBuilt']
df['Total Years Remodeled']=2020 - df['YearRemodAdd']
#Divided the total area into 4 groups in order to make the vizualization of the sizes easier. 
df['Range Area']=pd.qcut(df['Total Area'], q=4)

#total_area= df[['GrLivArea','LotArea', 'GarageArea']]
#total_area['Total Area']=total_area['GrLivArea']+total_area['LotArea']+ total_area['GarageArea']
#total_area.describe()

In [ ]:
f,ax=plt.subplots(figsize=(10,5))
sns.scatterplot(x='Total Area',y= 'SalePrice', data=df, hue='Neighborhood')
plt.legend(bbox_to_anchor=(1.05,1),loc=2)

In [ ]:
#Paolo: interesting plot, you see an overall trend price-area. Also the total area is in what units? Would be good
# to add it in the label of the plot to increase clarity.

In [ ]:
f, ax=plt.subplots(figsize=(20,5))
sns.barplot(x='Range Area',y= 'SalePrice',data=df)

~~~~
We can observe that smaller houses tend to be cheaper however the ones that are just
slighly bigger have a much higher price. Added to it, we can observe that houses in the area
of NoRidge tend to be quite expensive. 
~~~~

In [ ]:
f, ax=plt.subplots(figsize=(20,5))
sns.barplot(x='Neighborhood',y= 'SalePrice',data=df)

In [ ]:
#Paolo: yes, plot per area. here you can also try a boxplot. I have to say though that I find all these different
# colors quite distracting, why not a single color?

In [ ]:
f, ax=plt.subplots(figsize=(20,5))
sns.barplot(x='Neighborhood',y= 'SalePrice',hue='Range Area',data=df)
plt.legend(bbox_to_anchor=(1.05,1),loc=2)

~~~~
We can observe that houses in NoRidge, NridgHt and StoneBr tend to be the more expensive. While the houses in IDOTRR, Meadow and BrDale tend to be cheaper.  
~~~~

In [ ]:
f, ax=plt.subplots(figsize=(20,5))
sns.barplot(x='Total Rooms',y= 'SalePrice',data=df)

In [ ]:
f, ax=plt.subplots(figsize=(20,5))
sns.scatterplot(x='Total Rooms',y= 'SalePrice', hue='Total Bathrooms',data=df)

In [ ]:
df['Total Rooms without Bathrooms'] = df['Total Rooms']-df['Total Bathrooms']
f, ax=plt.subplots(figsize=(20,5))
sns.barplot(x='Total Rooms without Bathrooms',y= 'SalePrice',data=df)

In [ ]:
f, ax=plt.subplots(figsize=(20,5))
sns.barplot(x='Total Bathrooms',y= 'SalePrice',data=df)

~~~~
In the plots above we can observe that houses with a lower number of rooms tend to be cheaper while houses with a higher number of rooms tend to be more expensive. However, when we look only rooms (without counting bathrooms), houses with 1 room tend to be more expensive and when we look only bathrooms,  houses with 5 bathrooms tend to be more expensive. 
~~~~

In [ ]:
f,ax=plt.subplots(figsize=(10,5))
sns.scatterplot(x='Total Years Built', y = 'SalePrice', data=df)

In [ ]:
#Paolo: yes good plot, clear trend I would clarify the label in the plot like " numberf of years from build year" to make
# it more clear

In [ ]:
f,ax=plt.subplots(figsize=(10,5))
sns.scatterplot(x='Total Years Remodeled', y = 'SalePrice', data=df)

In [ ]:
#Paolo, yes same comment a above.

~~~~
We can observe a trend between the sale price and the number of years that have passed since the house was remodeled or built. The houses that were built or remodeled in the fewer years ago, tend to be more expensive while the ones that are older tend to be cheaper.  
~~~~

## Bonus Challenge 2 - Exploring Data with Correlation Heatmap

Now you have explored data visualizations with certain fields based on common sense. In the dataset there are many other fields that you are not sure whether they are important factors for the sale price. What is the best way to explore those fields without investigating them individually?

Making scatter matrix is not an option here because there are too many fields which makes it extremely time consuming to create scatter matrix. One option you have is to create a heatmap. Heatmaps are much less expensive to create than scatter matrixes. You can use heatmaps to visualize the pairwise correlations between each two variables.

Here is a [reference](https://seaborn.pydata.org/examples/many_pairwise_correlations.html) you can use to learn how to creat the pairwise correlation heatmap. Your heatmap should look like this [example](https://drive.google.com/file/d/1JhdNvbAnnWDFXEtDoBtx3B2KKIkqsnSH/view?usp=sharing)

In [ ]:
# your code here


In your heatmap, you can easily identify the highly correlated (either positively or negatively) variables by looking for the grids with darker colors. 

#### In the cell below, summarize what variables are highly correlated to the sale price?

In [ ]:
# your comment here


## Challenge 3 - Present Your Stories

Now based on your findings from the explorations, summarize and present your stories.

#### Present the top 5 factors that affect the sale price.

Use the following format to present each factor:

1. A title line about the factor.

1. No more than 3 sentences to describe the relationship between the factor and the sale price.

1. Support your point with the appropriate graph.

In [ ]:
# your responses here
# add cells as needed

#Location
#Houses located in NoRidge, NridgHt and StoneBr tend to be the more expensive. 
#While houses in IDOTRR, Meadow and BrDale tend to be cheaper.
f, ax=plt.subplots(figsize=(20,5))
sns.barplot(x='Neighborhood',y= 'SalePrice',hue='Range Area',data=df)
plt.legend(bbox_to_anchor=(1.05,1),loc=2)

In [ ]:
#Size
#Bigger houses tend to be more expensive.
f, ax=plt.subplots(figsize=(20,5))
sns.barplot(x='Range Area',y= 'SalePrice',data=df)

In [ ]:
#Years since built
#Older houses tend to be cheaper.
f,ax=plt.subplots(figsize=(10,5))
sns.scatterplot(x='Total Years Built', y = 'SalePrice', data=df)

In [ ]:
#Years since remodelled
#Houses that were remodelled and/or built more recently, tend to be more expensive.
f,ax=plt.subplots(figsize=(10,5))
sns.scatterplot(x='Total Years Remodeled', y = 'SalePrice', data=df)

In [ ]:
#Total number of rooms
#Houses with more rooms tend to be more expensive.
f, ax=plt.subplots(figsize=(20,5))
sns.barplot(x='Total Rooms',y= 'SalePrice',data=df)

In [ ]:
#Paolo: good work, you could try also boxplots